In [1]:
import pandas as pd
import geopandas as gpd
import shapely
import os
import json
from datetime import datetime, timedelta
import calendar

#### Add local files


In [2]:
working_dird = os.getcwd()
raw_data_folder = os.path.join(working_dird, 'data', 'raw')
output_dir = os.path.join(working_dird, 'data', 'output')
files = [os.path.join(raw_data_folder, x) for x in os.listdir(raw_data_folder)]

In [3]:
ct_dfs = [pd.read_csv(x) for x in files]

In [4]:
monthly_avg_price = ct_dfs[0]
monthly_reg_type = ct_dfs[1]
monthly_prod_sale = ct_dfs[2]
monthly_tax = ct_dfs[4]
approved_towns = ct_dfs[5]
shops = ct_dfs[6]

##### UNIFY PRODUCT TYPES


In [5]:
product_types_1 = list(monthly_reg_type['Product Type'].unique())
product_types_2 = list(monthly_prod_sale['Product Type'].unique())

product_types_2_dict = [product_types_2[-1], product_types_2[2], product_types_2[0], product_types_2[-2],
                        product_types_2[-1], product_types_2[-1], product_types_2[1], product_types_2[-1], product_types_2[-1]]

new_product_type_dict = {}

for x, y in list(zip(product_types_1, product_types_2_dict)):
    new_product_type_dict[x] = y
monthly_reg_type['Product Type'] = monthly_reg_type['Product Type'].apply(
    lambda x: new_product_type_dict[x])

##### LOCATIONS


In [6]:
shops['latitude'] = shops['Location'].apply(lambda x: shapely.from_wkt(x).y)
shops['longitude'] = shops['Location'].apply(lambda x: shapely.from_wkt(x).x)
shops['City'] = shops['City'].str.title()

In [7]:
usa_counties = gpd.read_file(
    "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_USA_2.json")

In [8]:
conntt = usa_counties[usa_counties['NAME_1'] == 'Connecticut']
conntt = conntt[['GID_1', 'NAME_1', 'NAME_2', 'geometry']]
conntt.columns = ['id', 'state', 'county', 'geometry']
conntt = conntt.reset_index()

In [9]:
def getIntersectionId(point: shapely.Point, bnds: list[shapely.MultiPolygon]):
    id = 0
    length = len(bnds)
    while (id < length):
        if (bnds[id].contains(point)):
            break
        id += 1
    return id

In [10]:
bnds = list(conntt['geometry'])
counties = list(conntt['county'])
shops['counties'] = shops['Location'].apply(
    lambda x: counties[getIntersectionId(shapely.from_wkt(x), bnds)])

In [11]:
shops = shops[['Type', 'Business', 'Street',
               'Zipcode', 'City', 'counties', 'latitude', 'longitude']]

In [12]:
shops.columns = ['type', 'business', 'street',
                 'zipcode', 'city', 'county', 'latitude', 'longitude']

##### NORMALISE DATES


In [40]:
monthly_prod_sale['month'] = monthly_prod_sale['Month Ending'].apply(lambda x: x.split(' ')[
    0])
monthly_prod_sale['year'] = monthly_prod_sale['Month Ending'].apply(
    lambda x: x.split(' ')[1])
monthly_avg_price['month'] = monthly_avg_price['Month Ending'].apply(lambda x: x.split(' ')[
    0])
monthly_avg_price['year'] = monthly_avg_price['Month Ending'].apply(
    lambda x: x.split(' ')[1])
monthly_reg_type['month'] = monthly_reg_type['Month'].apply(
    lambda x: x.split(' ')[0])
monthly_reg_type['year'] = monthly_reg_type['Month'].apply(
    lambda x: x.split(' ')[1])

In [42]:
monthly_tax = monthly_tax[['Month', 'Calendar Year', 'Fiscal Year',
                           'Plant Material Tax', 'Edible Products Tax', 'Other Cannabis Tax']]

In [43]:
monthly_tax.columns = ['month', 'year', 'fiscal_year',
                       'plant_material_tax', 'edible_products_tax', 'other_cannabis__tax']

In [55]:
def getEndofMonth(date: datetime):
    month = date.month
    year = date.year

    [first, days] = calendar.monthrange(year, month)

    endOfmonthDate: datetime = date + timedelta(days=days - 1)

    return endOfmonthDate.strftime("%Y-%m-%d")

In [56]:
monthly_tax["date"] = monthly_tax.apply(lambda x: getEndofMonth(
    datetime.strptime(f"{x.loc['year']}-{x.loc['month']}-01", "%Y-%B-%d")), axis=1)
monthly_prod_sale["date"] = monthly_prod_sale.apply(lambda x: getEndofMonth(
    datetime.strptime(f"{x.loc['year']}-{x.loc['month']}-01", "%Y-%B-%d")), axis=1)
monthly_avg_price["date"] = monthly_avg_price.apply(lambda x: getEndofMonth(
    datetime.strptime(f"{x.loc['year']}-{x.loc['month']}-01", "%Y-%B-%d")), axis=1)
monthly_reg_type["date"] = monthly_reg_type.apply(lambda x: getEndofMonth(
    datetime.strptime(f"{x.loc['year']}-{x.loc['month']}-01", "%Y-%B-%d")), axis=1)

C:\Users\User\AppData\Local\Temp\ipykernel_8412\1176593482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_tax["date"] = monthly_tax.apply(lambda x: getEndofMonth(datetime.strptime(f"{x.loc['year']}-{x.loc['month']}-01", "%Y-%B-%d")), axis=1)


##### OUTPUT DATA


In [57]:
monthly_tax.to_json(os.path.join(
    output_dir, 'ct_monthly_tax.json'), orient='records')
monthly_avg_price.to_json(os.path.join(
    output_dir, 'ct_monthly_avg_price.json'), orient='records')
monthly_prod_sale.to_json(os.path.join(
    output_dir, 'ct_monthly_prod_sale.json'), orient='records')
monthly_reg_type.to_json(os.path.join(
    output_dir, 'ct_monthly_reg_type.json'), orient='records')
shops.to_json(os.path.join(
    output_dir, 'ct_shops.json'), orient='records')

In [18]:
conntt.to_file(os.path.join(
    output_dir, 'ct_counties.geojson'), driver='GeoJSON')

##### Unified Data


In [3]:
monthly_tax = pd.read_json(os.path.join(
    output_dir, 'ct_monthly_tax.json'))
monthly_avg_price = pd.read_json(os.path.join(
    output_dir, 'ct_monthly_avg_price.json'))
monthly_prod_sale = pd.read_json(os.path.join(
    output_dir, 'ct_monthly_prod_sale.json'))
monthly_reg_type = pd.read_json(os.path.join(
    output_dir, 'ct_monthly_reg_type.json'))
shops = pd.read_json(os.path.join(
    output_dir, 'ct_shops.json'))

In [9]:
merge_date_data = pd.concat([monthly_prod_sale, monthly_tax, monthly_avg_price,
                            monthly_reg_type], axis=1, names=['date'], join='outer')

In [10]:
merge_date_data

,Month Ending,Product Type,Retail Sales Amount,Market,month,year,date,month,year,fiscal_year,...,Average Price Per Gram,month,year,date,Month,Product Type,Registrations,month,year,date
0,January 2023,Usable Cannabis (Flower),2819150.84,Adult-Use,January,2023,2023-01-31,August,2024.0,FY2024-25,...,12.32,January,2023.0,2023-01-31,January 2023,"Other (Pill, Capsule, Non Smokable Infused Ext...",7.0,January,2023.0,2023-01-31
1,January 2023,Vape Cartridge,721223.06,Adult-Use,January,2023,2023-01-31,May,2024.0,FY2023-24,...,11.73,February,2023.0,2023-02-28,January 2023,Infused Edible,9.0,January,2023.0,2023-01-31
2,January 2023,Infused Edible,688928.99,Adult-Use,January,2023,2023-01-31,April,2024.0,FY2023-24,...,10.35,March,2023.0,2023-03-31,January 2023,Usable Cannabis (Flower),109.0,January,2023.0,2023-01-31
3,January 2023,Extract for Inhalation,868829.67,Adult-Use,January,2023,2023-01-31,March,2024.0,FY2023-24,...,9.88,April,2023.0,2023-04-30,January 2023,Extract for Inhalation,87.0,January,2023.0,2023-01-31
4,January 2023,Usable Cannabis (Flower),4343508.44,Medical,January,2023,2023-01-31,February,2024.0,FY2023-24,...,9.77,May,2023.0,2023-05-31,January 2023,"Other (Pill, Capsule, Non Smokable Infused Ext...",4.0,January,2023.0,2023-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,December 2024,Usable Cannabis (Flower),3283933.34,Medical,December,2024,2024-12-31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT
212,December 2024,Vape Cartridge,2031459.28,Medical,December,2024,2024-12-31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT
213,December 2024,Infused Edible,841195.64,Medical,December,2024,2024-12-31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT
214,December 2024,Extract for Inhalation,370020.12,Medical,December,2024,2024-12-31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT


In [12]:
pd.concat([shops, merge_date_data], axis=1, names=[
          'Market', 'type']).to_csv('merge_data.csv')